In [45]:
import os 
import argparse
from numpy import nan
from biolib import liftover as liftover_utility
from liftover import get_lifter

In [46]:
lift_converter = get_lifter('hg38', 'hg19')

In [2]:
os.chdir('<project-dir>')

In [25]:
# commandline interface
fn = 'results/main/coloc/Results/eQTL_Catalogue/T1D_34012112_Gaulton/BLUEPRINT/T-cell/FINAL_Summary_Coloc_Gene_SNP_Pairs.bed'
outfn = 'test.tsv'
chr_col = 1 - 1
pos_col = 2 - 1
sep = '\t'
header = True

In [53]:
with open(fn) as fr, open(outfn, 'w') as fw:
    
    # write the header
    if header:
        header_str = next(fr)
        fw.write('{}{}{}'.format(header_str, sep, 'was_converted'))
        
    # parse the rest of the file
    for line in fr:
        
        # get chrom and pos
        info = line.strip().split(sep)
        chrom = info[chr_col]
        pos = int(info[pos_col])
        
        # get the new chrom and pos
        new_info = info.copy()
        new_coords = lift_converter[chrom][pos]
                 
        if len(new_coords) == 0:
            new_info[chr_col] = nan
            new_info[pos_col] = nan
            new_info.append(0)
            
        elif len(new_coords) == 1:
            new_info[chr_col] = new_coords[0][0]
            new_info[pos_col] = new_coords[0][1]
            new_info.append(1)

        else:
            raise Exception('Check this case: {}'.format(new_coords))
  
        new_info = [str(x) for x in new_info]
        new_info_str = sep.join(new_info) + '\n'
        fw.write(new_info_str)